In [1]:
import cv2
print(cv2.__version__)

import sys


4.10.0


In [2]:
#FOR ANOTHER PRODUCT COMMENT THE PRODUCT IMAGES GIVEN
#OUTPUT WILL BE SHOWN IN THE CURRENT DIRECTORY FOR THE IMAGES GIVEN

In [3]:
import numpy as np
import cv2

#To remove salt and pepper noise. using median filter to remove noice
def filterOutSaltPepperNoise(edgeImg):
    ctr = 0
    lastMedian = edgeImg
    median = cv2.medianBlur(edgeImg, 3)
    while not np.array_equal(lastMedian, median):
        # get those pixels that gets zeroed out
        zeroed = np.invert(np.logical_and(median, edgeImg))
        edgeImg[zeroed] = 0

        ctr = ctr + 1
        if ctr > 50:
            break
        lastMedian = median
        median = cv2.medianBlur(edgeImg, 3)

#to find edge contours
def findLargestContour(edgeImg):
    contours, hierarchy = cv2.findContours(edgeImg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #retr_external takes only extreme countors
    #chain_approx_simple returns the points that are only necessary for drawing contours
    
    #eliminating contours with smaller ares
    contoursWithArea = []
    for contour in contours:
        area = cv2.contourArea(contour)
        contoursWithArea.append([contour, area])

    contoursWithArea.sort(key=lambda tupl: tupl[1], reverse=True)
    largestContour = contoursWithArea[0][0]
    return largestContour


def back_ground_removal(img):
    
    #src = cv2.imread('b_org1.jpeg')
    src = img
    blurred = cv2.GaussianBlur(src, (5, 5), 0)

    blurred_float = blurred.astype(np.float32) / 255.0
    #This model does edge detection better with keeping much noises
    edgeDetector = cv2.ximgproc.createStructuredEdgeDetection("model.yml")
    #this fn returns 1 for edge, below we are doing *255 to get edge
    edges = edgeDetector.detectEdges(blurred_float) * 220.0
    
    #cv2.imwrite('edge-raw.jpg', edges)

    #converting the input into datatype of uint8
    edges_8u = np.asarray(edges, np.uint8)
    filterOutSaltPepperNoise(edges_8u)
    cv2.imwrite('edge.jpg', edges_8u)

    #after removing noise find edge countor
    contour = findLargestContour(edges_8u)
    # Draw the contour on the original image
    contourImg = np.copy(src)
    cv2.drawContours(contourImg, [contour], 0, (255, 0, 0), 2, cv2.LINE_AA, maxLevel=1)
    cv2.imwrite('contour.jpg', contourImg)

    #creating mask along edges
    mask = np.zeros_like(edges_8u)
    cv2.fillPoly(mask, [contour], 255)

    #dialating to calculate foreground
    mapFg = cv2.erode(mask, np.ones((5, 5), np.uint8), iterations=10)

    #estimating the foreground and background for grabcut
    trimap = np.copy(mask)
    trimap[mask == 0] = cv2.GC_BGD
    trimap[mask == 255] = cv2.GC_PR_BGD
    trimap[mapFg == 255] = cv2.GC_FGD

    #grabcut
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    rect = (0, 0, mask.shape[0] - 1, mask.shape[1] - 1)
    #bgmodel-temporary bg
    cv2.grabCut(src, trimap, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_MASK)

    # create mask again
    mask2 = np.where((trimap == cv2.GC_FGD) | (trimap == cv2.GC_PR_FGD), 255, 0).astype('uint8')
    
    #cv2.imwrite('mask2.jpg', mask2)

    #doing contour detection to avoid if holes/still background is present appear inside image
    contour2 = findLargestContour(mask2)
    mask3 = np.zeros_like(mask2)
    cv2.fillPoly(mask3, [contour2], 255)


    #to increase the intensity for smoothing the edges
    mask3 = np.repeat(mask3[:, :, np.newaxis], 3, axis=2)
    mask4 = cv2.GaussianBlur(mask3, (3, 3), 0)
    alpha = mask4.astype(float) * 1.1  # making blend stronger
    alpha[mask3 > 0] = 255.0
    alpha[alpha > 255] = 255.0

    #assigning the color values to foreground
    foreground = np.copy(src).astype(float)
    foreground[mask4 == 0] = 0
    background = np.ones_like(foreground, dtype=float) * 255.0

    #cv2.imwrite('foreground.png', foreground)
    #cv2.imwrite('background.png', background)
    #cv2.imwrite('alpha.png', alpha)

    # Normalize the alpha mask to keep intensity between 0 and 1. so we can multiply with foreround and foreground to make
    #it final.
    alpha = alpha / 255.0
    # Multiply the foreground with the alpha matte
    foreground = cv2.multiply(alpha, foreground)
    # Multiply the background with ( 1 - alpha )
    background = cv2.multiply(1.0 - alpha, background)
    # Add the masked foreground and background.
    cutout = cv2.add(foreground, background)

    #FINAL IMAGE AFTER REMOVING BACKGROUND
    #cv2.imwrite('cutout.jpg', cutout)
    return cutout



# Original Reference
# ref = cv2.imread("paste/paste_org.jpg")
ref = cv2.imread("bottle/b2.jpg")
print("B")

# Calling function
image_ref = back_ground_removal(ref)

# Saving reference after emoving background
cv2.imwrite("Org.jpg", image_ref)
# print(image_ref)


# Duplicate / image given by user
# src = cv2.imread("paste/paste_dup.jpg")
src = cv2.imread("bottle/bi3_dup.jpeg")
print("A")

# Calling function
image = back_ground_removal(src)

# Saving Image After removing background
cv2.imwrite("Dup.jpg", image)
# print(image)


cv2.imshow("org", image_ref)
cv2.imshow("dup", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

B
A


In [8]:
def image_cmp(image1, image1_ref):
    img_ref = image1_ref
    img = image1

    img_ref = cv2.resize(img_ref, (960, 540))
    img = cv2.resize(img, (960, 540))

    sift = cv2.xfeatures2d.SIFT_create()

    kp_1, desc_1 = sift.detectAndCompute(img_ref, None)
    kp_2, desc_2 = sift.detectAndCompute(img, None)

    # print("Key points 1st image " + str(len(kp_1)))
    # print("Key points 2nd image " + str(len(kp_2)))
    

    ''' Brute force method
    bf = cv2.BFMatcher(
        # cv2.NORM_L2, 
        # crossCheck=True
        )

    Match descriptors.
    matches = bf.match(des1,des2)
    matches=bf.knnMatch(desc_1, desc_2, k=2)
    '''

    # FLANN BASED MATCHING

    index_params = dict(algorithm=0, trees=5)
    search_params = dict()

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(desc_1, desc_2, k=2)

    # flann identifies nearest neighbours, knn srearch for k closet key points
    # It gives list with set of two points in it which contains feature vector of image1 and image2

    good_points = []

    for m, n in matches:
        if(m.distance < 0.85*n.distance):
            good_points.append(m)


    # print(len(matches))
    # print("Good matches " + str(len(good_points)))

    points = min(len(kp_1), len(kp_2))
    perc = len(good_points)/points * 100
    print("How good 2nd image is ", perc)

    result = cv2.drawMatches(img_ref, kp_1, img, kp_2, good_points, None)

    result = cv2.resize(result, (960, 540))

    cv2.imshow("result", result)
    cv2.imwrite("result.png", result)
    cv2.imshow("Ref", img_ref)
    cv2.imshow("Img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return perc

    
image = cv2.imread("Dup.jpg")
# image = cv2.resize(image, (960, 540))
image_ref = cv2.imread("Org.jpg")
# image_ref = cv2.resize(image_ref, (960, 540))

cmp_perc = image_cmp(image, image_ref)

if(cmp_perc>20):
    print("We have a similarity with the original image. We consider it as Original")
else:
    print("There is very low similarity with original image. We consider it as duplicate")

How good 2nd image is  8.557457212713937
There is very low similarity with original image. We consider it as duplicate
